## 1. Imports and config

In [1]:
A = "HISA-SMFM: HISTORICAL AND SENTIMENT ANALYSIS BASED STOCK MARKET FORECASTING MODEL".lower()
print(A)

hisa-smfm: historical and sentiment analysis based stock market forecasting model


In [1]:
import datetime as dt
import warnings
import yaml
import logging
import pywhatkit
import pandas as pd
from pmaw import PushshiftAPI
from tqdm import tqdm
from sqlalchemy import create_engine
from data_extraction_utils import data_prep_posts, join_submission_title_and_body, find_stock_symbols, data_prep_comments
warnings.filterwarnings("ignore")

In [2]:
# Specify logging settings
logging.basicConfig(filename='comments.log', level=logging.INFO, filemode="a", format="%(asctime)s - %(levelname)s - %(message)s")

## 2. Database configuration

In [3]:
# Reading form config.yaml"
with open("../../config.yaml", "r") as yamlconfig:
    config = yaml.load(yamlconfig, Loader=yaml.FullLoader)

# Create postgres string with db-config
postgres_username = config["db_config"]["postgres_username"]
postgres_password = config["db_config"]["postgres_password"]
postgres_address = config["db_config"]["postgres_address"]
postgres_port = config["db_config"]["postgres_port"]
postgres_dbname = config["db_config"]["postgres_dbname"]

postgres_str = f"postgresql://{postgres_username}:{postgres_password}@{postgres_address}:{postgres_port}/{postgres_dbname}"

# create db connection with sqlalchemy
cnx = create_engine(postgres_str)

## 3. Create stock symbol list

In [4]:
# load stock list and transform it into a set
stock_ticker_list = pd.read_excel("../../data/external/stock_ticker_list.xlsx")["symbol"].to_list()

# Modified second stock list with dollar sign
stock_ticker_list_with_dollar_sign = [f"${ticker}" for ticker in stock_ticker_list]

# Combine the two lists to one set
final_stock_ticker_list = set(stock_ticker_list + stock_ticker_list_with_dollar_sign)

## 4. Extract Comments from reddit

In [5]:
# Set date range to extract the data from the API
start = dt.datetime.strptime("04-08-2021", "%d-%m-%Y")
end = dt.datetime.strptime("05-08-2021", "%d-%m-%Y")
timestamp_list = [int((start + dt.timedelta(days=x)).timestamp()) for x in range(0, (end-start).days +1)]

# Reddit API Object
api = PushshiftAPI(num_workers=60, batch_size=60)

In [6]:

# loop over list of timestamps
for i in (pbar := tqdm(range(len(timestamp_list) -1))):

    # Set start date and end date
    start_date = timestamp_list[i]
    end_date = timestamp_list[i + 1]

    # Specify data to extract from reddit
    filters = ['author', 'created_utc', "score", "body"]

    # Call API function to retrieve data from reddit as a dataframe
    df = data_prep_comments(api, "wallstreetbets", start_time=start_date, end_time=end_date, filters=filters)

    # If response from API is empty -> Note that we have missing data for that day and continue
    if len(df) == 0:
        print(f"Missing Data for: {dt.datetime.fromtimestamp(start_date)}")
        logging.info(f"{df.shape[0]} rows written to DB - Date: {dt.datetime.fromtimestamp(start_date)}")
        pbar.set_description(f"{dt.datetime.fromtimestamp(start_date)}")
        continue

    # Transform timestamp into datetime column
    df["created_at"] = df["created_utc"].apply(lambda x: dt.datetime.fromtimestamp(x))

    # Get stock symbol from reddit post if available
    df["stock_symbol"] = df["body"].apply(lambda x: find_stock_symbols(x, final_stock_ticker_list))

    # Drop unused columns
    df.drop(columns=["created_utc"], inplace=True)

    # Rename columns as preparation for database
    df.rename(columns={'score': 'num_up_votes', "body": "post"}, inplace=True)

    # Write dataframe to database
    df.to_sql("r_wallstreetbets", cnx, index=False, if_exists="append")

    # Logging
    logging.info(f"{df.shape[0]} rows written to DB - Date: {dt.datetime.fromtimestamp(start_date)}")
    pbar.set_description(f"{dt.datetime.fromtimestamp(start_date)}")

  0%|          | 0/1 [09:25<?, ?it/s]


ValueError: A string literal cannot contain NUL (0x00) characters.

In [7]:
df

,author,post,num_up_votes,type,created_at,stock_symbol
0,EatMeatPeeps,"Quick, relate it to the market!",11,comment,2021-08-04 01:11:57,NaN
1,Trex_in_F16,De blasio more like de blasted,4,comment,2021-08-04 01:11:56,NaN
2,oscart57,Just stop please,1,comment,2021-08-04 01:11:54,NaN
3,Jtbny,I paid $12.75 per,3,comment,2021-08-04 01:11:53,NaN
4,1percentRolexWinner,Week?,1,comment,2021-08-04 01:11:51,NaN
...,...,...,...,...,...,...
44731,phoenixmusicman,We have to eject them 😡,1,comment,2021-08-04 23:16:31,NaN
44732,avelak,He's just trying to get people to accept his w...,1,comment,2021-08-04 23:16:31,NaN
44733,edibleblue,Oh yea saw them NYE Chicago... They opened the...,1,comment,2021-08-04 22:06:14,[AL]
44734,mMounirM,etsy is rekt,1,comment,2021-08-04 22:06:14,NaN


In [14]:
df.isnull().values

array([[False, False, False, False, False,  True],
       [False, False, False, False, False,  True],
       [False, False, False, False, False,  True],
       ...,
       [False, False, False, False, False, False],
       [False, False, False, False, False,  True],
       [False, False, False, False, False,  True]])

In [9]:
filters = ['author', 'created_utc', "score", "body"]

In [16]:
df["post"].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
44731    False
44732    False
44733    False
44734    False
44735    False
Name: post, Length: 44736, dtype: bool

In [10]:
posts = data_prep_comments(api, "wallstreetbets", start_time=1619820000, end_time=1619906400, filters=filters)

In [19]:
df[df["post"].isna()]

,author,post,num_up_votes,type,created_at,stock_symbol


In [14]:
posts

[{'author': 'Archdemon2212',
  'body': 'Due diligence',
  'created_utc': 1619822878,
  'score': 1},
 {'author': 'JimCramersCoke',
  'body': 'ocean water',
  'created_utc': 1619822871,
  'score': 4},
 {'author': 'Overdue_bills',
  'body': 'Puts on Monday.',
  'created_utc': 1619822869,
  'score': 1},
 {'author': 'Horror-Lemon3200',
  'body': 'Welcome! You found your home',
  'created_utc': 1619822867,
  'score': 2},
 {'author': 'get_MEAN_yall',
  'body': 'I can trade until 8pm eastern. What broker do you use?',
  'created_utc': 1619822866,
  'score': 0},
 {'author': 'dulkion',
  'body': 'And here I am preaching the GME gospel at Thanksgiving, and not buying in until January.  \n\nI need to listen to me more often.',
  'created_utc': 1619822866,
  'score': 2},
 {'author': 'SnooCauliflowers4003',
  'body': 'its the best public college system in the country?',
  'created_utc': 1619822864,
  'score': 1},
 {'author': 'Forevergogo',
  'body': "Oh man, I'd be selling TSLA so fast.  Everyone kn

In [15]:
pd.DataFrame(posts)

,author,body,created_utc,score
0,Archdemon2212,Due diligence,1619822878,1
1,JimCramersCoke,ocean water,1619822871,4
2,Overdue_bills,Puts on Monday.,1619822869,1
3,Horror-Lemon3200,Welcome! You found your home,1619822867,2
4,get_MEAN_yall,I can trade until 8pm eastern. What broker do ...,1619822866,0
...,...,...,...,...
95,fpcane305,Cool story Bro....lol,1619822679,1
96,fkgaslighters,This video made my day lmao,1619822679,1
97,Top_Caramel5267,Worlds are colliding,1619822674,2
98,letsgetapplebees,I like to show up on zoom in my pjs and beanie,1619822671,1


In [27]:
df[df["post"].str.contains("\x00")]

,author,post,num_up_votes,type,created_at,stock_symbol
14685,OrangePrestigious168,Give love to the growers and come to Scott's m...,2,comment,2021-08-04 20:10:01,NaN


In [30]:
df[df["post"].str.contains("\x00")]["post"]

14685    Give love to the growers and come to Scott's m...
Name: post, dtype: object

In [38]:
df[df["post"].str.contains("\x00")]["post"]

14685    Give love to the growers and come to Scott's m...
Name: post, dtype: object

In [42]:
df_cleaned = df.drop(df.loc[df["post"].str.contains("\x00")].index)

In [43]:
df_cleaned

,author,post,num_up_votes,type,created_at,stock_symbol
0,EatMeatPeeps,"Quick, relate it to the market!",11,comment,2021-08-04 01:11:57,NaN
1,Trex_in_F16,De blasio more like de blasted,4,comment,2021-08-04 01:11:56,NaN
2,oscart57,Just stop please,1,comment,2021-08-04 01:11:54,NaN
3,Jtbny,I paid $12.75 per,3,comment,2021-08-04 01:11:53,NaN
4,1percentRolexWinner,Week?,1,comment,2021-08-04 01:11:51,NaN
...,...,...,...,...,...,...
44731,phoenixmusicman,We have to eject them 😡,1,comment,2021-08-04 23:16:31,NaN
44732,avelak,He's just trying to get people to accept his w...,1,comment,2021-08-04 23:16:31,NaN
44733,edibleblue,Oh yea saw them NYE Chicago... They opened the...,1,comment,2021-08-04 22:06:14,[AL]
44734,mMounirM,etsy is rekt,1,comment,2021-08-04 22:06:14,NaN


In [44]:
df_cleaned.to_sql("r_wallstreetbets", cnx, index=False, if_exists="append")

735